In [1]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

document = [
#             'The phone has a good "screen".',
            '"iPod" is the best mp3 player.',
#             'Does the player play dvd with audio and "video"?',
#             'The camera is amazing and "easy" to use.',
#             'Canon "G3" has a great len.',
#             'If you want to buy a sexy, "cool", accessory-available mp3 player, you can choose iPod.'
           ]

# Dependency relations describing relations between opinion words and targets
# tak tambahi amod & nsubj
MR = ['mod', 'pnmod', 'subj', 's', 'obj', 'obj2','desc', 'amod', 'nsubj']
CONJ = ['conj', 'compound']

same_mod = ['mod', 'pnmod', 'amod']
same_bj = ['subj', 'obj', 'nsubj']

# Initial Opinion Words
opinion_words = [u'good', u'great', u'best', u'easy', u'cool']
ow_set = set(opinion_words)

# Initial Aspects (bukan dari paper, ini tambahan sendiri)
aspects = [u'video', u'G3']
a_set = set(aspects)


# Loop 1
for sentence in document:
    # Parse
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "pos, depparse, sentiment"})
    
#     print(result)
    for res in result['sentences']:
        print(res['parse'])
        # Rule 1.1
#         for data in res['basicDependencies']:
#             w1 = data['governorGloss']
#             w2 = data['dependentGloss']
#             w1_pos = res['tokens'][data['governor']-1]['pos']
#             w2_pos = res['tokens'][data['dependent']-1]['pos']
#             dep = data['dep']
            

(ROOT
  (S
    (NP (`` ``) (NN iPod) ('' ''))
    (VP (VBZ is)
      (NP (DT the) (JJS best) (NN mp3) (NN player)))
    (. .)))


In [2]:
import pandas as pd
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

custom_sent_tokenizer = PunktSentenceTokenizer()

def process_content(sentence):
    try:
        tokenized = custom_sent_tokenizer.tokenize(sentence)
        for i in tokenized:
            hasil = []
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            chunkGram = r"""Chunk: {<DT>?<NN.*|JJ>?<IN>?<PRP.*>?<NN.*>}"""
            chunkParser = nltk.RegexpParser(chunkGram)
            chunked = chunkParser.parse(tagged)
            for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
                panjang = len(subtree)
                rangkaian_kata = ''
                for i in range(0,panjang):
                    if i == 0:
                        rangkaian_kata += subtree[i][0]
                    else:
                        rangkaian_kata += ' ' + subtree[i][0]
                hasil.append(rangkaian_kata)
            return hasil

    except Exception as e:
        print(str(e))
        

print(process_content('"iPod" is the best mp3 player.'))

['iPod', 'mp3 player']
